# Asteroid Diameter Prediction Analysis and Modeling

This notebook contains the exploratory analysis, feature transformations,
and model training pipeline used in this project.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer, StandardScaler
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
import joblib

In [ ]:
asteroids_raw = pd.read_csv('dataset.csv')

In [ ]:
asteroids = asteroids_raw.copy().dropna(subset=['diameter'])
asteroids.info()

Renaming the features to be more self-descriptive.

In [ ]:
renamed_columns = {
  'class': 'class_',
  'pdes': 'designation',
  'neo': 'is_neo',
  'pha': 'is_pha',
  'H': 'absolute_magnitude',
  'e': 'eccentricity',
  'a': 'semi_major_axis',
  'q': 'perihelion_distance',
  'i': 'inclination',
  'om': 'longitude_of_ascending_node',
  'w': 'argument_of_perihelion',
  'ma': 'mean_anomaly',
  'ad': 'aphelion_distance',
  'n': 'mean_motion',
  'tp': 'perihelion_time',
  'tp_cal': 'perihelion_time_calendar',
  'per': 'period_in_days',
  'per_y': 'period_in_years',
  'moid': 'minimum_orbit_intersection_distance',
  'moid_ld': 'minimum_orbit_intersection_lunar_distance',
  'sigma_e': 'sigma_eccentricity',
  'sigma_a': 'sigma_semi_major_axis',
  'sigma_q': 'sigma_perihelion_distance',
  'sigma_i': 'sigma_inclination',
  'sigma_om': 'sigma_longitude_of_ascending_node',
  'sigma_w': 'sigma_argument_of_perihelion',
  'sigma_ma': 'sigma_mean_anomaly',
  'sigma_ad': 'sigma_aphelion_distance',
  'sigma_n': 'sigma_mean_motion',
  'sigma_tp': 'sigma_perihelion_time',
  'sigma_per': 'sigma_period_in_days'
}
asteroids.rename(columns=renamed_columns, inplace=True)

In [ ]:
asteroids.describe()

In [ ]:
asteroids.plot.box(column='diameter')

## Create Test Set

In [ ]:
train_set, test_set = train_test_split(asteroids, test_size=0.2, random_state=42)

In [ ]:
asteroids = train_set.copy()

## Explore Data

In [ ]:
corr_matrix = asteroids.corr(numeric_only=True)
corr_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
diameter_corr = corr_matrix['diameter'].sort_values(ascending=False)
diameter_corr

## Observations
* sigma perihelion time > 0.99 correlation with
sigma_argument_perihelion and sigma_mean_anomaly

* aphelion_distance > 0.99 correlation with semi_major_axis

* period_in_days > 0.99 correlation with semi_major_axis and aphelion_distance

* minimum_orbit_intersection_distance > 0.99 correlation with perihelion_distance

* None of the sigma values has > 0.3 absolute correlation with diameter

* The following variables have < 0.01 linear correlation with diameter

  1. longitude_of_ascending_node
  2. argument_of_perihelion
  3. mean_anomaly



In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(9, 3), sharey=True)
axs[0].scatter(x=asteroids['diameter'], y=asteroids['mean_anomaly'])
axs[0].set_ylabel('mean_anomaly')
axs[1].scatter(x=asteroids['diameter'], y=asteroids['argument_of_perihelion'])
axs[1].set_ylabel('argument_of_perihelion')
axs[2].scatter(x=asteroids['diameter'], y=asteroids['longitude_of_ascending_node'])
axs[2].set_ylabel('longitude_of_ascending_node')
fig.suptitle('Uncorrelated Features');

In [ ]:
asteroids[diameter_corr[:5].index.union(diameter_corr[-4:].index)].hist(figsize=(12, 9));

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 3))
axs[0].hist(asteroids['diameter'], bins=50)
axs[1].hist(np.log(asteroids['diameter']), bins=50)
fig.suptitle('Diameter vs Log-Transformed Diameter')

## Extract Features and Labels

In [ ]:
asteroids = train_set.drop(columns=['diameter'])
asteroids_labels = train_set['diameter']

## Clean data

In training data, *absolute_magnitude* has some null values. To fill those rows, we are going to use the median of all the other values using an imputer.

This won't only help us achieve a complete dataset this time, but protect us from missing features in new data.

In [ ]:
imputer = SimpleImputer(strategy="median")
asteroids_num = asteroids.select_dtypes(include=[np.number])
imputer.fit(asteroids_num)

In [ ]:
X = imputer.transform(asteroids_num)

In [ ]:
imputer.feature_names_in_

In [ ]:
asteroids.head()

## Handle Categorical Features

In [ ]:
cat_encoder = OneHotEncoder()
categorical_features = asteroids[['class_']]
cat_encoder.fit(categorical_features)
categorical_features.value_counts()

In [ ]:
asteroids_cat_encoder = cat_encoder.transform(categorical_features)

In [ ]:
ordinal_encoder = OrdinalEncoder()
categorical_features = asteroids[['is_neo', 'is_pha']]
ordinal_encoder.fit(categorical_features)

In [ ]:
asteroids_ordinal_encoder = ordinal_encoder.transform(categorical_features)

In [ ]:
std_scaler = StandardScaler()
asteroids_num_std_scaled = std_scaler.fit_transform(asteroids_num)

## Create Pipeline


In [ ]:
numeric_cols = [
    "aphelion_distance",
    "absolute_magnitude",
    "perihelion_distance",
    "inclination",
    "eccentricity",
    "mean_motion",
    "albedo"
]

In [ ]:
ordinal_transformer = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder()
)

onehot_transformer = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore")
)

numeric_transformer = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)

log_transformer = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log1p, feature_names_out="one-to-one"),
    StandardScaler()
)

preprocessor = make_column_transformer(
    (ordinal_transformer, ['is_neo', 'is_pha']),
    (onehot_transformer, ['class_']),
    (log_transformer, numeric_cols),
    remainder='drop'
)


In [ ]:
asteroids_preprocessed = preprocessor.fit_transform(asteroids)
asteroids_preprocessed_df = pd.DataFrame(
    asteroids_preprocessed,
    columns=preprocessor.get_feature_names_out(),
    index=asteroids.index)
asteroids_preprocessed_df.head()

## Train Models

In [ ]:
X_train = train_set.drop(columns=['diameter'])
X_test  = test_set.drop(columns=['diameter'])

y_train = np.log1p(train_set['diameter'])
y_test  = np.log1p(test_set['diameter'])

In [ ]:
lin_reg = make_pipeline(preprocessor, LinearRegression())
lin_reg.fit(X_train, y_train)

In [ ]:
ridge_model = make_pipeline(preprocessor, RidgeCV(alphas=np.logspace(-3, 3, 20)))
ridge_model.fit(X_train, y_train)

In [ ]:
lasso_model = make_pipeline(preprocessor, LassoCV(
        alphas=None,
        cv=5,
        max_iter=10_000,
        n_jobs=-1
    ))
lasso_model.fit(X_train, y_train)

In [ ]:
def evaluate(y_test_log, y_pred_log):
    return {
        "RMSE_log": np.sqrt(mean_squared_error(y_test_log, y_pred_log)),
        "R2_log": r2_score(y_test_log, y_pred_log),
        "RMSE": np.sqrt(mean_squared_error(
            np.expm1(y_test_log),
            np.expm1(y_pred_log)
        )),
        "MAE": mean_absolute_error(
            np.expm1(y_test_log),
            np.expm1(y_pred_log)
        )
    }

In [ ]:
models = {
    "Linear": lin_reg,
    "Ridge": ridge_model,
    "Lasso": lasso_model,
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_log = model.predict(X_test)
    results[name] = evaluate(y_test, y_pred_log)

pd.DataFrame(results).T


## Ship Final Model

In [ ]:
final_model = TransformedTargetRegressor(
    regressor=ridge_model,
    func=np.log1p,
    inverse_func=np.expm1
)


In [ ]:
final_model.fit(X_train, train_set['diameter'])

In [ ]:
joblib.dump(final_model, 'final_model.joblib')